In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url = "https://www.bestrandoms.com/random-address-in-in"

In [3]:
response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.content,'lxml')

In [6]:
ads = soup.findAll('li',class_ = 'col-sm-6')

In [7]:
ad = ads[0]

In [8]:
ad.get_text()

'Street:\xa0\xa02nd Fl, 114/8, Manek Bhavan, Caves Street, KalbadeviCity:\xa0\xa0 MumbaiState/province/area: \xa0\xa0 MaharashtraPhone number\xa0\xa002222056473Zip code\xa0\xa0400002Country calling code\xa0\xa0+91Country\xa0\xa0India'

In [9]:
ad.find('p').text.split(':')[1].strip()

'2nd Fl, 114/8, Manek Bhavan, Caves Street, Kalbadevi'

In [10]:
def get_dict_data(location_tag):
    dic = {}
    
    elements = location_tag.findAll('p')
    
    for e in elements[:3]:
        key = e.text.split(":")[0].strip()
        val = e.text.split(":")[1].strip()
        
        dic[key] = val
        
    z = elements[-2]
    key = z.text.split("\xa0")[0].strip()
    val = z.text.split("\xa0")[-1].strip()
    dic[key] = val
        
    return dic

In [11]:
get_dict_data(ad)

{'Street': '2nd Fl, 114/8, Manek Bhavan, Caves Street, Kalbadevi',
 'City': 'Mumbai',
 'State/province/area': 'Maharashtra',
 'Zip code': '400002'}

In [12]:
cols = ['Street','City','State/province/area','Zip code']

In [13]:
df = pd.DataFrame(columns = cols)

In [14]:
def main(df):
    while df.shape[0] < 1000:
        url = "https://www.bestrandoms.com/random-address-in-in"
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'lxml')
        
        ads = soup.findAll('li',class_ = 'col-sm-6')
        
        for ad in ads:
            data_dict = get_dict_data(ad)
            
            df = df.append(data_dict,ignore_index = True)
            
        print("{} rows added".format(df.shape[0]))
        
    return df

In [15]:
start = time.time()
final_df = main(df)
end = time.time()

print(round(end-start))

6 rows added
12 rows added
18 rows added
24 rows added
30 rows added
36 rows added
42 rows added
48 rows added
54 rows added
60 rows added
66 rows added
72 rows added
78 rows added
84 rows added
90 rows added
96 rows added
102 rows added
108 rows added
114 rows added
120 rows added
126 rows added
132 rows added
138 rows added
144 rows added
150 rows added
156 rows added
162 rows added
168 rows added
174 rows added
180 rows added
186 rows added
192 rows added
198 rows added
204 rows added
210 rows added
216 rows added
222 rows added
228 rows added
234 rows added
240 rows added
246 rows added
252 rows added
258 rows added
264 rows added
270 rows added
276 rows added
282 rows added
288 rows added
294 rows added
300 rows added
306 rows added
312 rows added
318 rows added
324 rows added
330 rows added
336 rows added
342 rows added
348 rows added
354 rows added
360 rows added
366 rows added
372 rows added
378 rows added
384 rows added
390 rows added
396 rows added
402 rows added
408 rows add

In [16]:
final_df['City'].value_counts()

Mumbai            528
Delhi             183
Bangalore          96
Hyderabad          52
Chennai            43
Ahmedabad          32
Vadodara           24
Pune               14
Kolkata            13
Coimbatore          8
Ludhiana            3
Lucknow             1
Srinagar            1
Vapi                1
Changanacherry      1
Jaipur              1
New Delhi           1
Name: City, dtype: int64

In [17]:
final_df.iloc[4]

Street                 Cccl Building, Jai Singh Road, Connaught Place
City                                                            Delhi
State/province/area                                             Delhi
Zip code                                                       110001
Name: 4, dtype: object

In [18]:
final_df['Street'].apply(lambda x : "Delhi" in x).sum()

0

In [19]:
def get_full_address(row):
    s = row['Street']
    c = row['City']
    p = row['Zip code']
    
    return s+", "+c+" "+p

In [20]:
final_df['Address'] = final_df.apply(get_full_address,axis=1)

In [21]:
final_df

,Street,City,State/province/area,Zip code,Address
0,"214, Arun Chambers, Tardeo Road, Tardeo",Mumbai,Maharashtra,400034,"214, Arun Chambers, Tardeo Road, Tardeo, Mumba..."
1,"30/1, 30/1, 1 Floor, Leemans Complex, Cunningh...",Bangalore,Karnataka,560052,"30/1, 30/1, 1 Floor, Leemans Complex, Cunningh..."
2,"V-33, Patel Nagar (w)",Delhi,Delhi,110008,"V-33, Patel Nagar (w), Delhi 110008"
3,"1, Sunawept Bldg, Lokhandwala Complex, Andheri(w)",Mumbai,Maharashtra,400053,"1, Sunawept Bldg, Lokhandwala Complex, Andheri..."
4,"Cccl Building, Jai Singh Road, Connaught Place",Delhi,Delhi,110001,"Cccl Building, Jai Singh Road, Connaught Place..."
...,...,...,...,...,...
997,"19, Swastik Plaza, Juhu",Mumbai,Maharashtra,400049,"19, Swastik Plaza, Juhu, Mumbai 400049"
998,"12-a Crystal Building, 36 Altamount Road, Cumb...",Mumbai,Maharashtra,400026,"12-a Crystal Building, 36 Altamount Road, Cumb..."
999,"71, 134/136, Empire Bldg, Dr D N Rd",Mumbai,Maharashtra,400001,"71, 134/136, Empire Bldg, Dr D N Rd, Mumbai 40..."
1000,"9, Jiten Building, Jp Road, Opp.rajkumar Hotel...",Mumbai,Maharashtra,400058,"9, Jiten Building, Jp Road, Opp.rajkumar Hotel..."


In [33]:
delhi_data = final_df.loc[final_df['City'] == 'Delhi']

In [42]:
from geopy.geocoders import Nominatim
nom = Nominatim(user_agent = 'tutorial',timeout = None)

In [41]:
def geocode_from_address(adr):
    
    location = None
    
    while location == None:
        location = nom.geocode(adr,timeout = None)
        if location != None:
            return (location.latitude,location.longitude)
        else:
            adr = ', '.join(adr.split(',')[1:])

In [44]:
start = time.time()
delhi_data['Geocodes'] = delhi_data['Address'].apply(lambda x: geocode_from_address(x))
end = time.time()
print(round(end - start))

258


C:\Users\lenovo\AppData\Local\Temp/ipykernel_4292/3479593779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delhi_data['Geocodes'] = delhi_data['Address'].apply(lambda x: geocode_from_address(x))


In [45]:
delhi_data

,Street,City,State/province/area,Zip code,Address,Latitude,Longitude,Geocodes
2,"V-33, Patel Nagar (w)",Delhi,Delhi,110008,"V-33, Patel Nagar (w), Delhi 110008",28.647259,77.165371,"(28.6472587, 77.1653712)"
4,"Cccl Building, Jai Singh Road, Connaught Place",Delhi,Delhi,110001,"Cccl Building, Jai Singh Road, Connaught Place...",NaN,NaN,"(28.6270183, 77.2153664)"
6,"3515, Chutani Manzil, Nicholson Road, Mori Gate",Delhi,Delhi,110006,"3515, Chutani Manzil, Nicholson Road, Mori Gat...",NaN,NaN,"(28.664203, 77.2237393)"
7,"20b, Dsidc Shed, Tilak Nagar",Delhi,Delhi,110018,"20b, Dsidc Shed, Tilak Nagar, Delhi 110018",NaN,NaN,"(28.6365482, 77.0964961)"
12,"32, Main, Lodhi Colony",Delhi,Delhi,110003,"32, Main, Lodhi Colony, Delhi 110003",NaN,NaN,"(28.586203, 77.2268576)"
...,...,...,...,...,...,...,...,...
978,M-1/a,Delhi,Delhi,110014,"M-1/a, Delhi 110014",28.651718,77.221939,"(28.6517178, 77.2219388)"
984,J 1/162,Delhi,Delhi,110027,"J 1/162, Delhi 110027",28.526923,77.256374,"(28.526923150000002, 77.25637382148926)"
985,"1520/4, Jagson Pal Building",Delhi,Delhi,110006,"1520/4, Jagson Pal Building, Delhi 110006",NaN,NaN,"(28.6517178, 77.2219388)"
991,"576/5, Ganda Nala Bazar, Hamilton Road, Kashme...",Delhi,Delhi,110006,"576/5, Ganda Nala Bazar, Hamilton Road, Kashme...",NaN,NaN,"(28.666814100000003, 77.22905486082311)"


In [46]:
delhi_data['Latitude'] = delhi_data['Geocodes'].apply(lambda x:x[0])
delhi_data['Longitude'] = delhi_data['Geocodes'].apply(lambda x:x[1])

C:\Users\lenovo\AppData\Local\Temp/ipykernel_4292/2210269239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delhi_data['Latitude'] = delhi_data['Geocodes'].apply(lambda x:x[0])
C:\Users\lenovo\AppData\Local\Temp/ipykernel_4292/2210269239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delhi_data['Longitude'] = delhi_data['Geocodes'].apply(lambda x:x[1])


In [47]:
delhi_data

,Street,City,State/province/area,Zip code,Address,Latitude,Longitude,Geocodes
2,"V-33, Patel Nagar (w)",Delhi,Delhi,110008,"V-33, Patel Nagar (w), Delhi 110008",28.647259,77.165371,"(28.6472587, 77.1653712)"
4,"Cccl Building, Jai Singh Road, Connaught Place",Delhi,Delhi,110001,"Cccl Building, Jai Singh Road, Connaught Place...",28.627018,77.215366,"(28.6270183, 77.2153664)"
6,"3515, Chutani Manzil, Nicholson Road, Mori Gate",Delhi,Delhi,110006,"3515, Chutani Manzil, Nicholson Road, Mori Gat...",28.664203,77.223739,"(28.664203, 77.2237393)"
7,"20b, Dsidc Shed, Tilak Nagar",Delhi,Delhi,110018,"20b, Dsidc Shed, Tilak Nagar, Delhi 110018",28.636548,77.096496,"(28.6365482, 77.0964961)"
12,"32, Main, Lodhi Colony",Delhi,Delhi,110003,"32, Main, Lodhi Colony, Delhi 110003",28.586203,77.226858,"(28.586203, 77.2268576)"
...,...,...,...,...,...,...,...,...
978,M-1/a,Delhi,Delhi,110014,"M-1/a, Delhi 110014",28.651718,77.221939,"(28.6517178, 77.2219388)"
984,J 1/162,Delhi,Delhi,110027,"J 1/162, Delhi 110027",28.526923,77.256374,"(28.526923150000002, 77.25637382148926)"
985,"1520/4, Jagson Pal Building",Delhi,Delhi,110006,"1520/4, Jagson Pal Building, Delhi 110006",28.651718,77.221939,"(28.6517178, 77.2219388)"
991,"576/5, Ganda Nala Bazar, Hamilton Road, Kashme...",Delhi,Delhi,110006,"576/5, Ganda Nala Bazar, Hamilton Road, Kashme...",28.666814,77.229055,"(28.666814100000003, 77.22905486082311)"


In [49]:
delhi_data['Latitude'].nunique()

106

In [50]:
delhi_data['Longitude'].nunique()

106

In [53]:
delhi_data = delhi_data.drop_duplicates(subset = ['Latitude'],keep = 'first')

In [57]:
delhi_data = delhi_data.reset_index(drop = True)

In [58]:
delhi_data.to_csv("Delhi Addresses.csv")

In [59]:
delhi_data

,Street,City,State/province/area,Zip code,Address,Latitude,Longitude,Geocodes
0,"V-33, Patel Nagar (w)",Delhi,Delhi,110008,"V-33, Patel Nagar (w), Delhi 110008",28.647259,77.165371,"(28.6472587, 77.1653712)"
1,"Cccl Building, Jai Singh Road, Connaught Place",Delhi,Delhi,110001,"Cccl Building, Jai Singh Road, Connaught Place...",28.627018,77.215366,"(28.6270183, 77.2153664)"
2,"3515, Chutani Manzil, Nicholson Road, Mori Gate",Delhi,Delhi,110006,"3515, Chutani Manzil, Nicholson Road, Mori Gat...",28.664203,77.223739,"(28.664203, 77.2237393)"
3,"20b, Dsidc Shed, Tilak Nagar",Delhi,Delhi,110018,"20b, Dsidc Shed, Tilak Nagar, Delhi 110018",28.636548,77.096496,"(28.6365482, 77.0964961)"
4,"32, Main, Lodhi Colony",Delhi,Delhi,110003,"32, Main, Lodhi Colony, Delhi 110003",28.586203,77.226858,"(28.586203, 77.2268576)"
...,...,...,...,...,...,...,...,...
101,"R-40, Vikas Marg Extn, Main Bus Stop, Shakarpur",Delhi,Delhi,110092,"R-40, Vikas Marg Extn, Main Bus Stop, Shakarpu...",28.630680,77.281898,"(28.6306797, 77.2818984)"
102,"3132, Ram Bazar",Delhi,Delhi,110006,"3132, Ram Bazar, Delhi 110006",28.651067,77.228135,"(28.651067, 77.2281349)"
103,"E 106, Jawahar Park, Sainik Farm",Delhi,Delhi,110062,"E 106, Jawahar Park, Sainik Farm, Delhi 110062",28.503746,77.216073,"(28.503746, 77.21607253402124)"
104,J 1/162,Delhi,Delhi,110027,"J 1/162, Delhi 110027",28.526923,77.256374,"(28.526923150000002, 77.25637382148926)"


In [62]:
error_geocode = delhi_data['Longitude'].max()

In [66]:
delhi_data = delhi_data.loc[delhi_data['Longitude'] != error_geocode]

In [68]:
delhi_data =  delhi_data.reset_index(drop = True)

In [69]:
delhi_data.to_csv("Delhi Addresses.csv")